In [1]:
!git clone https://github.com/nudojacopo/thesis.git

Cloning into 'thesis'...
remote: Enumerating objects: 1319, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 1319 (delta 24), reused 19 (delta 10), pack-reused 1273
Receiving objects: 100% (1319/1319), 61.81 MiB | 31.15 MiB/s, done.
Resolving deltas: 100% (845/845), done.


In [2]:
# Import and directories
root='/kaggle/working/'
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import random
from scipy.stats import chi2
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
!pip install powerlaw
import powerlaw
!pip install gdown
sys.path.append('/kaggle/working/thesis/src/PRO/PRO_package')
from functions import *

# Parameters to set up
platform='facebook_news'
format='csv'

# Link for data sources
link_for_drive={'facebook_brexit':'1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe','facebook_news':'1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB','voat':'1B7-YkPcjXcaqVa2wS8sA_fzx6lkBMEs9','reddit':'1QepHehlhqP-jtOcshFzajqxj1DpJtoj7','facebook_vaccines':'17q4X4JHYJBz1jF58XvYfbKqZLy-jtmzM','facebook_news':'1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB','gab':'1CpsRAaBVv4hIoq713KQmWYyZOlMG2BpH'}

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
import gdown
url='https://drive.google.com/uc?id='+link_for_drive[platform]
output='original_dataset.'+format
gdown.download(url,output,quiet=False)

if format=='csv':
    data=pd.read_csv('/kaggle/working/original_dataset.'+format)
if format=='parquet':
    data=pd.read_parquet('/kaggle/working/original_dataset.'+format)

Downloading...
From (original): https://drive.google.com/uc?id=1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB
From (redirected): https://drive.google.com/uc?id=1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB&confirm=t&uuid=47cb0bd0-29c5-4e7d-8f7c-dfc28325edc5
To: /kaggle/working/original_dataset.csv
100%|██████████| 1.84G/1.84G [00:13<00:00, 132MB/s] 


In [4]:
print(data.columns)

# Mandatory to check 
post_id = "post_id"
toxicity_score='toxicity_score'
comment_id='comment_id'
user='user_id'
created_at='created_at'
topic='topic'
social='social'
# Often to add
'''data['topic']='Brexit'
data['social']='Facebook'
data['created_at']=data['created_time']
'''

Index(['Unnamed: 0', 'user_id', 'from_name', 'message', 'created_at',
       'likes_count', 'comment_id', 'post_id', 'toxicity_score', 'topic',
       'social'],
      dtype='object')


"data['topic']='Brexit'\ndata['social']='Facebook'\ndata['created_at']=data['created_time']\n"

In [5]:
comment_counts = data[post_id].value_counts()
threads_with_more_than_50_comments = comment_counts[comment_counts > 50].index
data = data[data[post_id].isin(threads_with_more_than_50_comments)]
threads = data.groupby(post_id)  # Divide in threads

progress_bar = tqdm(total=len(threads), desc="Add variables...")
for name, group in threads:
    group['time'] = pd.to_datetime(group[created_at],format='ISO8601')
    group.sort_values(by=created_at, inplace=True)
    group['thread_birth'] = group['time'].min()
    group['temporal_distance_birth_h'] = round(pd.to_timedelta(group['time']-group['thread_birth']).dt.total_seconds() / 3600, 1)
    group['thread_lifetime_h'] = round((group['time'].max()-group['time'].min()).total_seconds() / 3600, 1)
    group['number_of_comments'] = len(group)
    group['number_of_users'] = len(group[user].unique())

    percentiles = add_percentile_column(group)['percentile']

    user_counts = group.groupby(user)[user].count()
    group['number_of_comments_by_user_in_thread'] = group[user].map(user_counts)
    group['user_appearances'] = group.groupby(user).cumcount() + 1

    data.loc[group.index, 'percentile'] = percentiles
    data.loc[group.index,'sequential_number_of_comment_by_user_in_thread'] = group['user_appearances']
    data.loc[group.index, 'number_of_comments_by_user_in_thread'] = group['number_of_comments_by_user_in_thread']
    data.loc[group.index, 'thread_birth'] = group['thread_birth']
    data.loc[group.index,'temporal_distance_birth_h'] = group['temporal_distance_birth_h']
    data.loc[group.index, 'thread_lifetime_h'] = group['thread_lifetime_h']
    data.loc[group.index, 'number_of_users'] = group['number_of_users']
    data.loc[group.index, 'number_of_comments'] = group['number_of_comments']
    progress_bar.update(1)

progress_bar.close()

data = data.sort_values(by=[user, created_at])
data[created_at] = pd.to_datetime(data[created_at],format='ISO8601')
data['temporal_distance_from_previous_comment_h'] = data.groupby(user)[created_at].diff().dt.total_seconds() / 3600
data = data.sort_values(by=[user, created_at])
data['IAT_user'] = data.groupby(user)[created_at].diff().dt.total_seconds()
data = data.sort_values(by=[user,post_id, created_at])
data['IAT_user_thread'] = data.groupby([user,post_id])[created_at].diff().dt.total_seconds()
data = data.sort_values(by=[post_id,created_at])
data['IAT_thread'] = data.groupby(post_id)[created_at].diff().dt.total_seconds()

data=data[[comment_id,user,post_id,  toxicity_score,
       created_at, 'percentile',
       'sequential_number_of_comment_by_user_in_thread',
       'number_of_comments_by_user_in_thread', 'thread_birth',
       'temporal_distance_birth_h', 'thread_lifetime_h', 'number_of_users', 'number_of_comments', 'temporal_distance_from_previous_comment_h',
       'IAT_user', 'IAT_user_thread', 'IAT_thread',topic,social]]
data.columns=['comment_id', 'user_id', 'post_id', 'toxicity_score', 'created_at',
       'percentile', 'sequential_number_of_comment_by_user_in_thread',
       'number_of_comments_by_user_in_thread', 'thread_birth',
       'temporal_distance_birth_h', 'thread_lifetime_h', 'number_of_users',
       'number_of_comments', 'temporal_distance_from_previous_comment_h',
       'IAT_user', 'IAT_user_thread', 'IAT_thread', 'topic', 'social']

Add variables...: 100%|██████████| 21401/21401 [15:51<00:00, 22.50it/s]


In [8]:
data.to_csv('/kaggle/working/'+platform+'_processed.csv', index=False)

In [9]:
data

,comment_id,user_id,post_id,toxicity_score,created_at,percentile,sequential_number_of_comment_by_user_in_thread,number_of_comments_by_user_in_thread,thread_birth,temporal_distance_birth_h,thread_lifetime_h,number_of_users,number_of_comments,temporal_distance_from_previous_comment_h,IAT_user,IAT_user_thread,IAT_thread,topic,social
1335565,944798945555598_944803585555134,10207300826412268,101911273177707_944798945555598,NaN,2015-12-24 20:50:45,0.39,1.0,1.0,2015-12-24 20:50:45,0.0,24.0,252.0,257.0,NaN,NaN,NaN,NaN,News,Facebook
1335374,944798945555598_944806012221558,10154039612859590,101911273177707_944798945555598,0.033589,2015-12-24 20:57:33,0.78,1.0,1.0,2015-12-24 20:50:45,0.1,24.0,252.0,257.0,NaN,NaN,NaN,408.0,News,Facebook
1335536,944798945555598_944807432221416,10205188220699313,101911273177707_944798945555598,0.011454,2015-12-24 21:03:07,1.17,1.0,1.0,2015-12-24 20:50:45,0.2,24.0,252.0,257.0,NaN,NaN,NaN,334.0,News,Facebook
1335464,944798945555598_944807918888034,1051136788281211,101911273177707_944798945555598,0.008712,2015-12-24 21:05:14,1.56,1.0,1.0,2015-12-24 20:50:45,0.2,24.0,252.0,257.0,NaN,NaN,NaN,127.0,News,Facebook
1335323,944798945555598_944808305554662,1351132308235801,101911273177707_944798945555598,0.389771,2015-12-24 21:06:48,1.95,1.0,1.0,2015-12-24 20:50:45,0.3,24.0,252.0,257.0,6242.818889,22474148.0,NaN,94.0,News,Facebook
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6642528,743960822290654_22995404,1198941196806671,99597577059_743960822290654,0.025349,2014-03-23 17:59:01,92.16,1.0,1.0,2014-03-23 05:48:27,12.2,15.1,35.0,51.0,NaN,NaN,NaN,5376.0,News,Facebook
6642541,743960822290654_22995755,10154334588579399,99597577059_743960822290654,0.022630,2014-03-23 19:32:48,94.12,4.0,5.0,2014-03-23 05:48:27,13.7,15.1,35.0,51.0,6.857778,24688.0,24688.0,5627.0,News,Facebook
6642527,743960822290654_22995760,10154334588579399,99597577059_743960822290654,0.595890,2014-03-23 19:34:04,96.08,5.0,5.0,2014-03-23 05:48:27,13.8,15.1,35.0,51.0,0.021111,76.0,76.0,76.0,News,Facebook
6642540,743960822290654_22995768,857124677754399,99597577059_743960822290654,0.333410,2014-03-23 19:37:30,98.04,1.0,1.0,2014-03-23 05:48:27,13.8,15.1,35.0,51.0,571.768333,2058366.0,NaN,206.0,News,Facebook
